In [1]:
import z3
import sys
from z3 import *
z3.set_param(proof=True)
import typing
import operator
sys.path.append("../../hw/base")
#from ..basics.asm_interp import AsmInterp
from verification_utils import CHCs, mk_int_array


stack = Array('stack', IntSort(), IntSort())
sp = Int('sp')
r0, r1 = Ints('r0 r1')
mem = Array('mem', IntSort(), IntSort())
state_vars = [stack, sp, r0, r1, mem]
Invariant: typing.TypeAlias = typing.Callable[[state_vars], bool]


def enumerateLabels(prog) -> dict:
    labels = {}
    for i, op in enumerate(prog):
        if isinstance(op, tuple):
            continue
        # label:
        labels[op[:-1]] = i
    return labels

def createCHCs(precond, postcond, prog):
    
    U = {i: Function(f"U{i}", *(v.sort() for v in [*state_vars]), BoolSort())
     for i in range(len(prog)+3)}
    l = [Implies(precond(state_vars), U[0](state_vars))]
    i = 0

    
    labels = enumerateLabels(prog)
    
    for op in prog:
        if isinstance(op, str): # this is a label, so we skip it
            continue

        SINGLE_OPERAND_OPS = {'STOR', 'LOAD', 'HALT'}
        OPTIONAL_OPERAND_OPS = {'DUP', 'POP'}

        if op[0] in SINGLE_OPERAND_OPS:
            if op[0] == "LOAD":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, mem[r0]), sp+1, r0, r1, mem)))
            elif op[0] == "STOR":
                l.append(Implies(U[i](state_vars), U[i+1](stack, sp, r0, r1, Store(mem, r1, r0))))
            elif op[0] == "HALT":
                break
            i += 1
            continue

        if op[0] in OPTIONAL_OPERAND_OPS: #TODO: verify this + remove dup and pop from the code below.
            if op[0] == "DUP":
                n = int(op[1]) if len(op) > 1 else 0
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, stack[sp-1-n]), sp+1, r0, r1, mem)))
            elif op[0] == "POP":
                n = int(op[1]) if len(op) > 1 else 1
                if n == 2:
                    l.append(Implies(U[i](state_vars), U[i+1](stack, sp-2, stack[sp-2], stack[sp-1], mem)))
                else: # n == 1
                    l.append(Implies(U[i](state_vars), U[i+1](stack, sp-1, stack[sp-1], r1, mem)))
            i += 1
            continue
        # two arguments
        op1, op2 = op
        
        if op1 == 'PUSH':
            l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, int(op2)), sp+1, r0, r1, mem)))
        elif op1 == "ALU":
            if op2 == "ADD":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 + r1), sp+1, r0, r1, mem)))
            elif op2 == "MUL":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 * r1), sp+1, r0, r1, mem)))
            elif op2 == "SUB":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 - r1), sp+1, r0, r1, mem)))
            elif op2 == "NEG":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, -r0), sp+1, r0, r1, mem)))
            elif op2 == "AND":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 & r1), sp+1, r0, r1, mem)))
            elif op2 == "OR":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 | r1), sp+1, r0, r1, mem)))
            elif op2 == "NOT":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, ~r0), sp+1, r0, r1, mem)))
            elif op2 == "SHL":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 << 1), sp+1, r0, r1, mem)))
            elif op2 == "SHR":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 >> 1), sp+1, r0, r1, mem)))
            elif op2 == "LT":
                l.append(Implies(And(U[i](state_vars), r0 < r1), U[i+1](Store(stack, sp, 1), sp+1, r0, r1, mem)))
                l.append(Implies(And(U[i](state_vars), r1 <= r0), U[i+1](Store(stack, sp, 0), sp+1, r0, r1, mem)))
        elif op1 == "JMP":
            l.append(Implies(U[i](state_vars), U[labels[op2]](state_vars)))
        elif op1 == "JNZ":
            l.append(Implies(And(U[i](state_vars), r0 != 0), U[labels[op2]](state_vars)))
            l.append(Implies(And(U[i](state_vars), r0 == 0), U[i+1](state_vars)))
        elif op1 == "JZ":
            l.append(Implies(And(U[i](state_vars), r0 == 0), U[labels[op2]](state_vars)))
            l.append(Implies(And(U[i](state_vars), r0 != 0), U[i+1](state_vars)))
        elif op1 == "RET": # r0 = 0 
            l.append(Implies(And(U[i](state_vars), op2 == 1), U[state_vars[2]](Store(stack, sp, r1), sp+1, r0, r1, mem)))
            l.append(Implies(And(U[i](state_vars), op2 != 1), U[state_vars[2]](state_vars)))
        i += 1
    l.append(Implies(U[i](state_vars), postcond(state_vars)))

    chcs = CHCs(l)
    
    return chcs

In [ ]:
prog = [
    ('POP', 1), # u0 => u1
    ('STOR', ), # u1 => u2
    ('PUSH', 1), # u2 => u3
    ('POP', 2), # u3 => u4
    ('STOR', ), # u4 => u5
    'loop:', # u[5] => u[6](sdfsdf)
    ('PUSH', 0), # u5 => u6
    ('POP', 1), # u6 => u7
    ('LOAD', ),
    ('PUSH', 1),
    ('POP', 2),
    ('ALU', 'SUB'),
    ('PUSH', 0),
    ('POP', 2),
    ('STOR', ),
    ('JZ', 'end'),
    ('PUSH', 2),
    ('POP', 2),
    ('STOR', ), # r1 = 2, r0 = 40
    ('PUSH', 1),
    ('POP', 1),
    ('LOAD', ), # on stack = current max
    ('PUSH', 2),
    ('POP', 1),
    ('LOAD', ), # on stack = [currentMax, current]
    ('POP', 2), # r1 = current, r0 = currentMax
    ('ALU', 'LT'),
    ('POP', 1),
    ('JZ', 'loop'),
    ('PUSH', 2),
    ('POP', 1),
    ('LOAD', ),
    ('PUSH', 1),
    ('POP', 2), # r1 = 1
    ('STOR', ),
    ('JMP', 'loop'),
    'end:',
    ('PUSH', 1),
    ('POP', 1),
    ('LOAD', ),
    ('POP', 1)
    ]
expected = [1,2,3,4,5,5]

precond = lambda state_vars: And(And(And(And([Select(state_vars[0], i) == expected[i] for i in range(len(expected))]), state_vars[1] == 0), state_vars[2] == 0), state_vars[3] == 0)
postcond = lambda state_vars: r0 == 5
chcs = createCHCs(precond, postcond, prog)
chcs.solve   ({'xform.inline_eager': False,
   'xform.inline_linear': False})



In [2]:

prog1 = [
    ('PUSH', 5),
    ('PUSH', 5),
    ('POP', 2),  # r0 = 5, r1 = 5
    ('ALU', 'LT'),  # Should push 0
    ('POP', 1), # r0 = 0
    ('STOR', ), # mem[5] = 0
    ('PUSH', 5),
    ('POP', 1), # r0 = 5
    ('LOAD', ) # stack[sp] = mem[5] = 0
]

precond1 = lambda state_vars: True
postcond1 = lambda state_vars: Select(state_vars[0], state_vars[1]-1) == 0 # Check if r0 == 1
chcs1 = createCHCs(precond1, postcond1, prog1)
chcs1.solve()


U0,True
U1,"∃k!3 : ν1 = Store(k!3, -1 + ν2, 5)"
U8,"(∃k!5, k!4 : ν6 = Store(k!4, 5, 0) ∧ ν4 = ν2[ν3] ∧ ν2 = Store(Store(Store(k!5, ν3, 5), 1 + ν3, 5), ν3, 5) ∧ ν5 = 5) ∨ (∃k!3, k!2 : (∃k!4, k!3 : ν4 = Store(Store(Store(k!4, -1 + k!3, 5), k!3, 5), -1 + k!3, 5) ∧ ν8 = Store(k!3, 5, 0) ∧ k!2 = 0 ∧ ν7 = 5) ∧ ν4 = ν2[-1 + k!3] ∧ ν3 = -1 + k!3)"
U5,"(∃k!4 : ν3 = ν1[ν2] ∧ ν1 = Store(Store(Store(k!4, ν2, 5), 1 + ν2, 5), ν2, 0) ∧ ν4 = 5) ∨ (∃k!3, k!2 : ((∃k!4 : k!2 = Store(Store(k!4, -1 + k!3, 5), k!3, 5)[-1 + k!3] ∧ ¬(5 ≤ k!2) ∧ ν3 = Store(Store(Store(k!4, -1 + k!3, 5), k!3, 5), -1 + k!3, 1) ∧ ν6 = 5) ∨ (∃k!4 : k!2 = Store(Store(k!4, -1 + k!3, 5), k!3, 5)[-1 + k!3] ∧ 5 ≤ k!2 ∧ ν3 = Store(Store(Store(k!4, -1 + k!3, 5), k!3, 5), -1 + k!3, 0) ∧ ν6 = 5)) ∧ ν4 = ν2[-1 + k!3] ∧ ν3 = -1 + k!3)"
U7,"(∃k!4, k!3 : ν2 = Store(Store(Store(k!4, -1 + ν3, 5), ν3, 5), -1 + ν3, 5) ∧ ν6 = Store(k!3, 5, 0) ∧ ν4 = 0 ∧ ν5 = 5) ∨ (∃k!4, k!3 : (∃k!5, k!4 : ν8 = Store(k!4, 5, ν6) ∧ ν6 = k!4[k!3] ∧ k!4 = Store(Store(Store(k!5, k!3, 5), 1 + k!3, 5), k!3, 0) ∧ ν7 = 5) ∧ ν3 = 1 + k!3 ∧ ν2 = Store(k!4, k!3, 5))"
U4,"(∃k!4 : ν3 = Store(Store(k!4, -1 + ν2, 5), ν2, 5)[-1 + ν2] ∧ ¬(5 ≤ ν3) ∧ ν1 = Store(Store(Store(k!4, -1 + ν2, 5), ν2, 5), -1 + ν2, 1) ∧ ν4 = 5) ∨ (∃k!4 : ν3 = Store(Store(k!4, -1 + ν2, 5), ν2, 5)[-1 + ν2] ∧ 5 ≤ ν3 ∧ ν1 = Store(Store(Store(k!4, -1 + ν2, 5), ν2, 5), -1 + ν2, 0) ∧ ν4 = 5) ∨ (∃k!4, k!3 : (∃k!5 : ν5 = k!4[k!3] ∧ ν6 = k!4[1 + k!3] ∧ k!4 = Store(Store(k!5, k!3, 5), 1 + k!3, 5)) ∧ ν3 = 1 + k!3 ∧ ν5 ≤ ν4 ∧ ν2 = Store(k!4, k!3, 0)) ∨ (∃k!4, k!3 : (∃k!5 : ν5 = k!4[k!3] ∧ ν6 = k!4[1 + k!3] ∧ k!4 = Store(Store(k!5, k!3, 5), 1 + k!3, 5)) ∧ ν3 = 1 + k!3 ∧ ¬(ν5 ≤ ν4) ∧ ν2 = Store(k!4, k!3, 1))"
U3,"(∃k!5 : ν3 = ν1[ν2] ∧ ν4 = ν1[1 + ν2] ∧ ν1 = Store(Store(k!5, ν2, 5), 1 + ν2, 5)) ∨ (∃k!3 : (∃k!5 : ν2 = Store(Store(k!5, -2 + k!3, 5), -1 + k!3, 5)) ∧ ν2 = -2 + k!3 ∧ ν3 = ν1[-2 + k!3] ∧ ν4 = ν1[-1 + k!3])"
U9,"(∃k!4, k!3 : ν2 = Store(Store(Store(k!4, -1 + ν3, 5), ν3, 5), -1 + ν3, ν6[5]) ∧ ν6 = Store(k!3, 5, 0) ∧ ν4 = 5 ∧ ν5 = 5) ∨ (∃k!4, k!3 : (∃k!5, k!4 : ν8 = Store(k!4, 5, 0) ∧ ν6 = k!4[k!3] ∧ k!4 = Store(Store(Store(k!5, k!3, 5), 1 + k!3, 5), k!3, 5) ∧ ν7 = 5) ∧ ν3 = 1 + k!3 ∧ ν2 = Store(k!4, k!3, ν6[ν4]))"
U6,"(∃k!5, k!4 : ν6 = Store(k!4, 5, ν4) ∧ ν4 = ν2[ν3] ∧ ν2 = Store(Store(Store(k!5, ν3, 5), 1 + ν3, 5), ν3, 0) ∧ ν5 = 5) ∨ (∃k!0 : (∃k!4 : ν4 = ν2[ν3] ∧ ν2 = Store(Store(Store(k!4, ν3, 5), 1 + ν3, 5), ν3, 0) ∧ ν5 = 5) ∧ ν5 = Store(k!0, ν4, ν3))"
U2,"(∃k!5 : ν1 = Store(Store(k!5, -2 + ν2, 5), -1 + ν2, 5)) ∨ (∃k!4, k!3 : (∃k!3 : k!4 = Store(k!3, -1 + k!3, 5)) ∧ ν3 = 1 + k!3 ∧ ν2 = Store(k!4, k!3, 5))"


In [9]:
prog1 = [
    ('PUSH', 5),
    ('DUP', ),
    ('POP', 2)
]

precond1 = lambda state_vars: And(And(state_vars[1]==0, state_vars[2]==0), state_vars[3]==0)
postcond1 = lambda state_vars: And(state_vars[2] == 5, state_vars[3] == 5) # Check if r0 == 5 
chcs1 = createCHCs(precond1, postcond1, prog1)
chcs1.solve()



unsat